In [1]:
import sys
sys.path.insert(0, '../')
sys.path.insert(0, '../../')

import numpy as np
import os
import timeit
%autosave 0

Autosave disabled


In [2]:
from queries import Queries
from queries import concatenate
from queries import find_constant_features
from click_simulator import DependentClickModel

In [9]:
# Load Queries
queries_list = []
for name in ['train.txt', 'test.txt', 'vali.txt']:
    data_dir = '../../datasets/MSLR10K/Fold1/' + name
    queries = Queries.load_from_text(data_dir)
    cls = find_constant_features(queries)
    queries.adjust(remove_features=cls, purge=True, scale=True)
    queries.save(data_dir[:-3]+'dat')
    queries_list.append(queries)
query_all = concatenate(queries_list)
query_all.save('../../datasets/MSLR10K.dat')

In [28]:
data_dir = '../../../datasets/MSLR10K.dat'
queries = Queries.load(data_dir)

In [11]:
simulator = DependentClickModel('perfect')

In [12]:
start = timeit.default_timer()
n_clicks = 0
n_docs = 0
iteration = int(1e5)
for i in range(iteration):
    query = queries[np.random.choice(len(queries))]
    while query.document_count() < 10 or query.relevance_scores.max() == 0:
        query = queries[np.random.choice(len(queries))]
    y =  query.relevance_scores[np.random.permutation(query.document_count())]
    click = simulator.get_click(y)
    n_clicks += sum(click)
    n_docs += len(click)

print("running time of %d sessions: %f" %(iteration, timeit.default_timer()-start))
print("number of clicks: %d" % n_clicks)
print("number of docs: %d" % n_docs)

running time of 100000 sessions: 68.644067
number of clicks: 96598
number of docs: 12403485


In [16]:
import lightgbm as gbm

/Users/lichang/Documents/Repositories/OnlineDiverseLTR/venv/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [19]:
t_query = queries[:2]
v_query = queries[2:4]

In [36]:
ranker_params = {
    'min_child_samples': 50,
    'min_child_weight': 0,
    'n_estimators': 500,
    'learning_rate': 0.02,
    'num_leaves': 400,
    'boosting_type': 'gbdt',
    'objective': 'lambdarank'
}
fit_params = {
    'early_stopping_rounds': 50,
    'eval_metric': 'ndcg',
    'eval_at': 5,
    'verbose': 5,
}


In [32]:
train_features = t_query.feature_vectors
train_labels = t_query.relevance_scores
train_qid = np.diff(t_query.query_indptr)
valid_features = v_query.feature_vectors
valid_labels = v_query.relevance_scores
valid_qid = np.diff(v_query.query_indptr)

In [33]:
train_features.shape

(151, 136)

In [34]:
train_qid

array([93, 58], dtype=int32)

In [44]:
ranker = gbm.LGBMRanker(**ranker_params)
ranker.fit(X=train_features, y=train_labels, group=train_qid,
           eval_set=[(valid_features, valid_labels)], eval_group=[valid_qid],
           **fit_params)

Training until validation scores don't improve for 50 rounds
[5]	valid_0's ndcg@5: 0.267772
[10]	valid_0's ndcg@5: 0.326724
[15]	valid_0's ndcg@5: 0.35787
[20]	valid_0's ndcg@5: 0.27023
[25]	valid_0's ndcg@5: 0.27023
[30]	valid_0's ndcg@5: 0.27023
[35]	valid_0's ndcg@5: 0.27023
[40]	valid_0's ndcg@5: 0.27023
[45]	valid_0's ndcg@5: 0.27023
[50]	valid_0's ndcg@5: 0.27023
Early stopping, best iteration is:
[1]	valid_0's ndcg@5: 0.5455


LGBMRanker(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
           importance_type='split', learning_rate=0.02, max_depth=-1,
           min_child_samples=50, min_child_weight=0, min_split_gain=0.0,
           n_estimators=500, n_jobs=-1, num_leaves=400, objective='lambdarank',
           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)